In [82]:
import pandas as pd
import numpy as np

import folium
from folium.plugins import AntPath

from pprint import pprint
from pprint import PrettyPrinter

In [83]:
df_intercambio = pd.read_csv('../Obtencion datos/intercambio_electrico.csv')

df_intercambio = df_intercambio.drop(columns=['Unnamed: 0'])

df_intercambio

,nombre,tipo de intercambio,Valores,Porcentaje,Fecha actualización,dia,mes,año
0,Francia,Exportación,-8063.400,0.120426,30/09/2024,30,9,2024
1,Francia,Exportación,-17724.900,0.274826,01/10/2024,1,10,2024
2,Francia,Exportación,-34403.500,0.539259,02/10/2024,2,10,2024
3,Francia,Importación,58894.000,0.879574,30/09/2024,30,9,2024
4,Francia,Importación,46770.100,0.725174,01/10/2024,1,10,2024
5,Francia,Importación,29394.233,0.460741,02/10/2024,2,10,2024
6,Francia,saldo,50830.600,1.000000,30/09/2024,30,9,2024
7,Francia,saldo,29045.200,1.000000,01/10/2024,1,10,2024
8,Francia,saldo,-5009.267,1.000000,02/10/2024,2,10,2024
9,Portugal,Exportación,-64814.700,0.910008,30/09/2024,30,9,2024


In [84]:
df_intercambio.columns

Index(['nombre', 'tipo de intercambio', 'Valores', 'Porcentaje',
       'Fecha actualización', 'dia', 'mes', 'año'],
      dtype='object')

In [85]:
df_intercambio['nombre'].unique()

array(['Francia', 'Portugal', 'Marruecos', 'Andorra'], dtype=object)

In [86]:
df_intercambio['tipo de intercambio'].unique()

array(['Exportación', 'Importación', 'saldo'], dtype=object)

In [87]:
españa_alt = 40.4637
españa_lat = -3.7492

In [98]:
#Creacion del mapa
spain_map = folium.Map(location = [españa_alt, españa_lat], 
                       zoom_start= 5, 
                       tiles = 'Esri Worldimagery', 
                       width='600px', 
                       height='1000px'
                       )

spain_map


In [96]:
#Coordenadas de los paises donde existe un intercambio energetico.
lista_paises = ['Marruecos', 'Francia', 'Portugal', 'Andorra', 'España']
lista_altitudes = [31.83999, 46.57771, 39.68023, 42.5462, 40.4637]
lista_latitudes = [-6.19721, 2.78159, -8.80606, 1.5034, -3.7492]


df_coordenadas = pd.DataFrame()
df_coordenadas['nombre'] = lista_paises
df_coordenadas['altitud'] = lista_altitudes
df_coordenadas['latitud'] = lista_latitudes

df_coordenadas

,nombre,altitud,latitud
0,Marruecos,31.83999,-6.19721
1,Francia,46.57771,2.78159
2,Portugal,39.68023,-8.80606
3,Andorra,42.54620,1.50340
4,España,40.46370,-3.74920


In [90]:
marruecos = [df_coordenadas['altitud'][0], df_coordenadas['latitud'][0]]
francia = [df_coordenadas['altitud'][1], df_coordenadas['latitud'][1]]
portugal = [df_coordenadas['altitud'][2], df_coordenadas['latitud'][2]]
andorra = [df_coordenadas['altitud'][3], df_coordenadas['latitud'][3]]
españa = [df_coordenadas['altitud'][4], df_coordenadas['latitud'][4]]

In [91]:
df_unido = pd.merge(df_intercambio, df_coordenadas, how = 'left', on = 'nombre')


In [92]:
df_unido

,nombre,tipo de intercambio,Valores,Porcentaje,Fecha actualización,dia,mes,año,altitud,latitud
0,Francia,Exportación,-8063.400,0.120426,30/09/2024,30,9,2024,46.57771,2.78159
1,Francia,Exportación,-17724.900,0.274826,01/10/2024,1,10,2024,46.57771,2.78159
2,Francia,Exportación,-34403.500,0.539259,02/10/2024,2,10,2024,46.57771,2.78159
3,Francia,Importación,58894.000,0.879574,30/09/2024,30,9,2024,46.57771,2.78159
4,Francia,Importación,46770.100,0.725174,01/10/2024,1,10,2024,46.57771,2.78159
5,Francia,Importación,29394.233,0.460741,02/10/2024,2,10,2024,46.57771,2.78159
6,Francia,saldo,50830.600,1.000000,30/09/2024,30,9,2024,46.57771,2.78159
7,Francia,saldo,29045.200,1.000000,01/10/2024,1,10,2024,46.57771,2.78159
8,Francia,saldo,-5009.267,1.000000,02/10/2024,2,10,2024,46.57771,2.78159
9,Portugal,Exportación,-64814.700,0.910008,30/09/2024,30,9,2024,39.68023,-8.80606


In [93]:
intercambios = folium.map.FeatureGroup(name='Intercambios')

input_tipo_inter = input('Introduce el tipo de intercambio: importación, exportación, o saldo: ______ ').lower()

df_filtrado = df_unido[df_unido['tipo de intercambio'].str.lower() == input_tipo_inter]


for lat, lng, pais, valores, porcentaje, fecha in zip(df_filtrado['altitud'], 
                           df_filtrado['latitud'], 
                           df_filtrado['nombre'],  
                           df_filtrado['Valores'], 
                           df_filtrado['Porcentaje'],
                           df_filtrado['Fecha actualización']):

        contenido_label = f'''<b> Pais: {pais} </b><br> 
                            <b>Tipo de intercambio: {input_tipo_inter} </b><br> 
                            <b>Valores: {valores} </b><br> 
                            <b>Porcentaje: {porcentaje} </b><br>
                            <b>Fecha actualización: {fecha} </b>'''
        intercambios.add_child(folium.Marker(location=[lat, lng], 
                                        popup=contenido_label))



spain_map.add_child(intercambios)


folium.Marker(
        location= españa,
        icon=folium.DivIcon(
                html = '<div style="font-size: 50px;" >🇪🇦</div>'),
        popup = 'España'
        ).add_to(spain_map)

folium.Marker(
        location= portugal,
        icon=folium.DivIcon(
                html = '<div style="font-size: 30px;" >🇵🇹</div>'),
        popup = 'Portugal'
        ).add_to(spain_map)
folium.Marker(
        location= francia,
        icon=folium.DivIcon(
                html = '<div style="font-size: 30px;" >🇫🇷</div>'),
        popup = 'Francia'
        ).add_to(spain_map)
folium.Marker(
        location= marruecos,
        icon=folium.DivIcon(
                html = '<div style="font-size: 30px;" >🇲🇦</div>'),
        popup = 'Marruecos'
        ).add_to(spain_map)
folium.Marker(
        location= andorra,
        icon=folium.DivIcon(
                html = '<div style="font-size: 30px;" >🇦🇩</div>'),
        popup = 'Andorra'
        ).add_to(spain_map)





AntPath(locations = [marruecos, españa],
        color = 'red',
        delay = 1000,
        weight = 5).add_to(spain_map)

AntPath(locations = [francia, españa],
        color = 'blue',
        delay = 2000,
        weight = 5).add_to(spain_map)

AntPath(locations = [andorra, españa],
        color = 'yellow',
        delay = 500,
        weight = 5).add_to(spain_map)

AntPath(locations = [portugal, españa],
        color = 'green',
        delay = 4000,
        weight = 5).add_to(spain_map)

spain_map

In [94]:
# TO-DO;
# CUANDO SEA EXPORTACIÓN - ANTPATH SEA DE ESPAÑA AL PAÍS EXPORTADO.
# CUANDO SEA IMPORTACIÓN - ANTPATH SEA DEL PAIS DE IMPORTACION A ESPAÑA.
# CUANDO SEA SALDO - SI EL NUMERO ES POSITIVO, QUE SEA DE DE FUERA A DENTRO, SI ES NEGATIVO, QUE SEA DE DENTRO A FUERA.


#CREACIÓN MAPA CU